In [ ]:
import qualified Data.ByteString.Lazy.Char8 as L8
import qualified Data.ByteString.Lazy as L
import Data.Char (isSpace, chr)
import Data.Int (Int64)
import Data.Word (Word8)

In [ ]:
data ParseState = ParseState {
    string :: L.ByteString
  , offset :: Int64
  } deriving (Show)

newtype Parse a = Parse {
    runParse :: ParseState -> Either String (a, ParseState)
  }

In [ ]:
getState :: Parse ParseState
getState = Parse (\s -> Right (s, s))

putState :: ParseState -> Parse ()
putState s = Parse (\_ -> Right ((), s))

In [ ]:
identity :: a -> Parse a
identity a = Parse (\s -> Right (a, s))

In [ ]:
bail :: String -> Parse a
bail err = Parse $ \s -> Left $
    "byte offset " ++ show (offset s) ++ ": " ++ err

In [ ]:
(==>) :: Parse a -> (a -> Parse b) -> Parse b
firstParser ==> secondParser = Parse chainedParser
    where 
        chainedParser initState = case runParse firstParser initState of
            Left errMsg -> Left errMsg
            Right (firstResult, newState) ->
                runParse (secondParser firstResult) newState

In [ ]:
identity :: a -> Parse a
identity a = Parse (\s -> Right (a, s))

In [ ]:
instance Functor Parse where
    fmap f parser = parser ==> \result ->
        identity (f result)

In [ ]:
parse :: Parse a -> L.ByteString -> Either String a
parse parser initState
    = case runParse parser (ParseState initState 0) of
        Left err -> Left err
        Right (result, _) -> Right result

In [ ]:
identity :: a -> Parse a
identity a = Parse (\s -> Right (a, s))

In [ ]:
parseByte :: Parse Word8
parseByte = getState ==> \initState ->
    case L.uncons (string initState) of
        Nothing -> bail "no more input"
        Just (byte, remainder) -> putState newState ==> \_ -> identity byte
            where
                newState = initState { string = remainder
                                                      , offset = newOffset }
                newOffset = offset initState + 1

In [ ]:
w2c :: Word8 -> Char
w2c = chr . fromIntegral

parseChar :: Parse Char
parseChar = w2c <$> parseByte

peekByte :: Parse (Maybe Word8)
peekByte = (fmap fst . L.uncons . string) <$> getState

peekChar :: Parse (Maybe Char)
peekChar = fmap w2c <$> peekByte

parseWhile :: (Word8 -> Bool) -> Parse [Word8]
parseWhile p = (fmap p <$> peekByte) ==>